# Feed Neural Networks Assignment
### Thennakoon TMBSB 219412A

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit
import sklearn.metrics as metrics
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.utils import plot_model

### Dataset

In [5]:
df = pd.read_csv("drive/My Drive/MSc/Semester 05/NN/Assignment 01/dataset/crx.data",sep=',', 
    header=None)
df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-
689,b,35.00,3.375,u,g,c,h,8.29,f,f,0,t,g,00000,0,-


In [6]:
df.dtypes

0      object
1      object
2     float64
3      object
4      object
5      object
6      object
7     float64
8      object
9      object
10      int64
11     object
12     object
13     object
14      int64
15     object
dtype: object

In [7]:
df.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64

In [8]:
cat_col = [0,1, 3, 4, 5, 6, 8, 9, 11, 12, 13, 15]
df[cat_col] = df[cat_col].astype('category')
df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-
689,b,35.00,3.375,u,g,c,h,8.29,f,f,0,t,g,00000,0,-


In [10]:
df.dtypes

0     category
1     category
2      float64
3     category
4     category
5     category
6     category
7      float64
8     category
9     category
10       int64
11    category
12    category
13    category
14       int64
15    category
dtype: object

In [11]:
for col in cat_col:
    df[col] = df[col].cat.codes

In [12]:
df.tail()
df.dtypes

0        int8
1       int16
2     float64
3        int8
4        int8
5        int8
6        int8
7     float64
8        int8
9        int8
10      int64
11       int8
12       int8
13      int16
14      int64
15       int8
dtype: object

### Normalizing Input and OutPuts

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df))

df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
count,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000
mean,0.830435,0.431315,0.169954,0.738647,0.488889,0.476605,0.666023,0.078014,0.523188,0.427536,0.035821,0.457971,0.088406,0.349369,0.010174,0.555072
std,0.254597,0.275613,0.177792,0.159029,0.286709,0.308590,0.288278,0.117422,0.499824,0.495080,0.072581,0.498592,0.278934,0.283716,0.052101,0.497318
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,0.203438,0.035714,0.666667,0.333333,0.142857,0.444444,0.005789,0.000000,0.000000,0.000000,0.000000,0.000000,0.135294,0.000000,0.000000
50%,1.000000,0.382521,0.098214,0.666667,0.333333,0.428571,0.888889,0.035088,1.000000,0.000000,0.000000,0.000000,0.000000,0.305882,0.000050,1.000000
75%,1.000000,0.647564,0.257411,0.666667,0.333333,0.785714,0.888889,0.092105,1.000000,1.000000,0.044776,1.000000,0.000000,0.564706,0.003955,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
X= df.iloc[:,0:15].values   #0:15
Y= df.iloc[:,15:16].values

In [17]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, Y, test_size=0.3, random_state=42)

In [18]:
np.random.seed(42)

In [ ]:
# from sklearn.model_selection import train_test_split 
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [19]:
sFolds = StratifiedKFold(n_splits =5)

### Model Parameters

In [20]:
depth = 3
no_of_neurons = [15, 10, 1]
activation_func = ['relu', 'relu', 'sigmoid']
#loss_func = 'binary_crossentropy'
loss_func = 'mse'
regularizer =  tf.keras.regularizers.l2(0) 

### Generate Model

In [21]:
def generate_model():
  model = tf.keras.models.Sequential()
  print('Network configuration ',no_of_neurons)

  model.add(tf.keras.layers.Dense(no_of_neurons[0], input_dim=15, activation = activation_func[0], kernel_regularizer=regularizer))

  for x in range(1, depth-1):
      model.add(tf.keras.layers.Dense(no_of_neurons[x], activation = activation_func[x],kernel_regularizer=regularizer))  

  model.add(tf.keras.layers.Dense(no_of_neurons[depth-1], activation = activation_func[depth-1]))  
    
  model.compile(loss = loss_func , optimizer = 'adam' , metrics = ['binary_accuracy']) 
        
  return model

### Evaluate Model

In [22]:
def evaluate(verbose=False):
    n_split=5
    f1_scores = []
    losses = []

    for train_index,test_index in StratifiedKFold(n_split).split(X_train1, y_train1):      # StratifiedKFold, KFold
        x_train,x_test=X[train_index],X[test_index]
        y_train,y_test=Y[train_index],Y[test_index]

        model=generate_model()
        model.fit(x_train, y_train,epochs=100, verbose=0)
        evaluationMetrics = model.evaluate(x_test,y_test, verbose=0)
        
        if verbose:
            print('Model evaluation ',evaluationMetrics)   # This returns metric values for the evaluation

        y_pred = np.where(model.predict(x_test) > 0.5, 1, 0)
        f1 = metrics.f1_score(y_test, y_pred , average="macro")

        if verbose:
            print('F1 score is ', f1)
        
        f1_scores.append(f1)
        losses.append(evaluationMetrics[0])
    
    return np.mean(f1_scores), np.mean(losses)

In [36]:
evaluate()

Network configuration  [15, 10, 1]
Network configuration  [15, 10, 1]
Network configuration  [15, 10, 1]
Network configuration  [15, 10, 1]
Network configuration  [15, 10, 1]


(0.6049439274072677, 0.11606017127633095)

In [26]:
x = [[15,10,1],[15, 25, 1 ], [30,15,1]]
for i in x:
  depth = 3
  no_of_neurons = i
  #activation_func = ['relu', 'relu', 'sigmoid']
  activation_func = ['tanh', 'tanh', 'sigmoid']
  loss_func = 'binary_crossentropy'
  #loss_function = 'mse'
  regularizer =  tf.keras.regularizers.l2(0) 

  print(evaluate())

Network configuration  [15, 10, 1]
Network configuration  [15, 10, 1]
Network configuration  [15, 10, 1]
Network configuration  [15, 10, 1]
Network configuration  [15, 10, 1]
(0.7089160848243337, 0.38818862587213515)
Network configuration  [15, 25, 1]
Network configuration  [15, 25, 1]
Network configuration  [15, 25, 1]
Network configuration  [15, 25, 1]
Network configuration  [15, 25, 1]
(0.694275928668074, 0.3995523348450661)
Network configuration  [30, 15, 1]
Network configuration  [30, 15, 1]
Network configuration  [30, 15, 1]
Network configuration  [30, 15, 1]
Network configuration  [30, 15, 1]
(0.6953131763765766, 0.3919758588075638)


In [30]:
x = [[15,1],[30,1]]
for i in x:
  depth = 2
  no_of_neurons = i
  #activation_func = ['relu', 'sigmoid']
  activation_func = ['tanh', 'sigmoid']
  loss_func = 'binary_crossentropy'
  #loss_function = 'mse'
  regularizer =  tf.keras.regularizers.l2(0) 

  print(evaluate())

Network configuration  [15, 1]
Network configuration  [15, 1]
Network configuration  [15, 1]
Network configuration  [15, 1]
Network configuration  [15, 1]
(0.7032908205422926, 0.3911636799573898)
Network configuration  [30, 1]
Network configuration  [30, 1]
Network configuration  [30, 1]
Network configuration  [30, 1]
Network configuration  [30, 1]
(0.6984211886436951, 0.3925758108496666)


In [34]:
x = [[15,20,10,1],[15,20,30,1], [15,10, 5,1], [15,15,15,1]]
for i in x:
  depth = 4
  no_of_neurons = i
  #activation_func = ['relu', 'relu', 'relu',  'sigmoid']
  activation_func = ['tanh','tanh','tanh', 'sigmoid']
  loss_func = 'binary_crossentropy'
  #loss_function = 'mse'
  regularizer =  tf.keras.regularizers.l2(0) 

  print(evaluate())

Network configuration  [15, 20, 10, 1]
Network configuration  [15, 20, 10, 1]
Network configuration  [15, 20, 10, 1]
Network configuration  [15, 20, 10, 1]
Network configuration  [15, 20, 10, 1]
(0.7228106698698589, 0.40755943208932877)
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
(0.7104736912060875, 0.3827498838305473)
Network configuration  [15, 10, 5, 1]
Network configuration  [15, 10, 5, 1]
Network configuration  [15, 10, 5, 1]
Network configuration  [15, 10, 5, 1]
Network configuration  [15, 10, 5, 1]
(0.7030742483436161, 0.40171498209238055)
Network configuration  [15, 15, 15, 1]
Network configuration  [15, 15, 15, 1]
Network configuration  [15, 15, 15, 1]
Network configuration  [15, 15, 15, 1]
Network configuration  [15, 15, 15, 1]
(0.7220642965874238, 0.3984789505600929)


## Regularizer related testing

In [36]:
x = [0.001, 0.01, 0.1, 1]
for i in x:
  depth = 4
  no_of_neurons = [15,20,30,1]
  activation_func = ['relu', 'relu', 'relu',  'sigmoid']
  #activation_func = ['tanh','tanh','tanh', 'sigmoid']
  loss_func = 'binary_crossentropy'
  #loss_function = 'mse'
  regularizer =  tf.keras.regularizers.l2(i) 

  print(evaluate())

Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
(0.6289370523062827, 0.46975400745868684)
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
(0.693710073861407, 0.4783938616514206)
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
(0.2861118213013224, 0.7696570992469788)
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
(0.14323705124108202, 0.8149792909622192)


In [37]:
x = [0.001, 0.01, 0.1, 1]
for i in x:
  depth = 4
  no_of_neurons = [15,20, 30, 1]
  activation_func = ['relu', 'relu', 'relu',  'sigmoid']
  #activation_func = ['tanh','tanh', 'tanh', 'sigmoid']
  loss_func = 'binary_crossentropy'
  #loss_function = 'mse'
  regularizer =  tf.keras.regularizers.l1(i) 

  print(evaluate())

Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
(0.7089212997256522, 0.48995672166347504)
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
(0.6725024967801914, 0.5327632188796997)
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
(0.14323705124108202, 0.8349617838859558)
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
Network configuration  [15, 20, 30, 1]
(0.14323705124108202, 0.9601915717124939)


In [38]:
depth = 4
no_of_neurons = [15,20,30,1]
activation_func = ['relu', 'relu', 'relu',  'sigmoid']
#activation_func = ['tanh','tanh','tanh', 'sigmoid']
loss_func = 'binary_crossentropy'
  #loss_function = 'mse'
regularizer =  tf.keras.regularizers.l1(0.001) 

In [39]:
model=generate_model()
model.fit(X_train1, y_train1,epochs=50, verbose=0)
evaluationMetrics = model.evaluate(X_test1,y_test1, verbose=0)
y_pred = np.where(model.predict(X_test1) > 0.5, 1, 0)
f1 = metrics.f1_score(y_test1, y_pred , average="macro")

Network configuration  [15, 20, 30, 1]


In [40]:
f1

0.8405648267008985

In [41]:
evaluationMetrics[0]

0.45473170280456543